In [1]:
import tensorflow as tf

#### 1. 生成文件存储样例数据。

In [2]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 模拟海量数据将数据写入不同文件
num_shards = 2             # 总共写入多少个文件
instances_per_shard = 2    # 每个文件中有多少个数据
for i in range(num_shards):
    # 数据被分为多个文件时，将不同文件以类似 0000n-of-0000m 的后缀区分
    # m:总共多少文件，n:当前文件的编号
    filename = ('Records/data.tfrecords-%.5d-of-%.5d' % (i, num_shards)) 
    
    # 将数据封装成Example结构写入TFRecord文件。
    writer = tf.python_io.TFRecordWriter(filename)
    for j in range(instances_per_shard):
        # Example结构仅包含当前样例属于第几个文件以及是当前文件的第几个样本。
        example = tf.train.Example(features=tf.train.Features(feature={
            'i': _int64_feature(i),
            'j': _int64_feature(j)}))
        writer.write(example.SerializeToString())
    writer.close()  

#### 2. 读取文件。
从文件中读取单个样例

In [3]:
# 获取文件列表
files = tf.train.match_filenames_once("Records/data.tfrecords-*")

# 创建输入队列，shuffle为false，不打乱文件列表顺序
filename_queue = tf.train.string_input_producer(files, shuffle=False) 

# 读取并解析一个example
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
      serialized_example,
      features={
          'i': tf.FixedLenFeature([], tf.int64),
          'j': tf.FixedLenFeature([], tf.int64),
      })
with tf.Session() as sess:
    # 初始化变量
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    print(sess.run(files))
    
    # 声明Coordinator类来协同不同线程，并启动线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 执行6次获取数据的操作
    for i in range(6):
        print(sess.run([features['i'], features['j']]))
    coord.request_stop()
    coord.join(threads)

[b'Records/data.tfrecords-00000-of-00002'
 b'Records/data.tfrecords-00001-of-00002']
[0, 0]
[0, 1]
[1, 0]
[1, 1]
[0, 0]
[0, 1]


#### 3. 组合训练数据（Batching）
读取batch样例

In [6]:
# i表示样例，j表示特征
example, label = features['i'], features['j']
# 一个batch中样例的个数
batch_size = 2
# 队列中可以存储的样例个数
capacity = 1000 + 3 * batch_size
# tf.train.batch来组合样例
# 还有tf.train.shuffle_batch，该函数多一个参数，限制出队时队列中的元素的最少个数
#example_batch, label_batch = tf.train.batch([example, label], batch_size=batch_size, capacity=capacity)
example_batch, label_batch = tf.train.shuffle_batch([example, label], batch_size=batch_size, capacity=capacity,
                                                   min_after_dequeue=30)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    # 获取并打印组合之后的样例
    # 真是问题中，该输出一般作为神经网络的输入
    for i in range(3):
        cur_example_batch, cur_label_batch = sess.run([example_batch, label_batch])
        print(cur_example_batch, cur_label_batch)
    coord.request_stop()
    coord.join(threads)


[0 1] [1 0]
[1 0] [1 1]
[0 1] [0 1]
